## Relayed HOPR tokens by the ct-app

In [ ]:
import requests
from dotenv import load_dotenv
import os

In [ ]:
# Load environment variables
load_dotenv('env_var.env')

# Access environment variables
subgraph_url = os.environ['SUBGRAPH_URL']
api_host = os.environ['API_HOST']
api_key = os.environ['API_KEY']

if 'SUBGRAPH_URL' in os.environ and 'API_HOST' in os.environ and 'API_KEY' in os.environ:
    print(True)
else:
    print(False)

In [ ]:
def get_subgraph_data():
    """
    This function retrieves safe_address-node_address-balance links from the
    specified subgraph using pagination.
    """
    query = """
            query SafeNodeBalance($first: Int, $skip: Int) {
                safes(first: $first, skip: $skip) {
                    registeredNodesInNetworkRegistry {
                    node {
                        id
                    }
                    safe {
                        id
                        balance {
                        wxHoprBalance
                        }
                    }
                    }
                }
            }
        """

    data = {
        "query": query,
        "variables": {
            "first": 1000,
            "skip": 0,
        },
    }
    subgraph_dict = {}
    more_content_available = True
    pagination_skip_size = 1000

    while more_content_available:
        try:
            response = requests.post(subgraph_url, json=data)

            if response.status_code != 200:
                print(f"Received status code {response.status_code} when querying The Graph API")
                break

            json_data = response.json()

        except requests.exceptions.RequestException:
            print("An error occurred while sending the request to subgraph endpoint")
            return {}
        except ValueError:
            print("An error occurred while parsing the response as JSON from subgraph endpoint")
            return {}
        except Exception:
            print("An unexpected error occurred")
            return {}

        safes = json_data["data"]["safes"]
        for safe in safes:
            for node in safe["registeredNodesInNetworkRegistry"]:
                node_address = node["node"]["id"]
                wxHoprBalance = node["safe"]["balance"]["wxHoprBalance"]
                safe_address = node["safe"]["id"]
                subgraph_dict[node_address] = {
                    "safe_address": safe_address,
                    "wxHOPR_balance": wxHoprBalance,
                }

        # Increment skip for the next iteration
        data["variables"]["skip"] += pagination_skip_size
        more_content_available = len(safes) == pagination_skip_size

    return subgraph_dict

def get_unique_nodeAddress_peerId_aggbalance_links(api_host, api_key):
    """
    Returns a dict containing all unique source_peerId-source_address links.
    """
    channel_url = "http://{}:3001/api/v3/channels/?includingClosed=false&fullTopology=true".format(api_host)
    headers = {'X-Auth-Token': api_key}
    response = requests.request("GET", channel_url, headers=headers)

    if response.status_code != 200:
        print("Could not fetch channel information. Status code: {}".format(response.status_code))
        return {}

    response = response.json()

    if 'all' not in response:
            print("Response does not contain `all`")
            return {}

    peerid_address_aggbalance_links = {}
    for item in response["all"]:
        if "sourcePeerId" not in item or "sourceAddress" not in item:
            print("Response does not contain `source_peerid` or `source_address`")
            continue

        if "status" not in item:
            print("Response does not contain `status`")
            continue

        source_peer_id = item["sourcePeerId"]
        source_address = item["sourceAddress"]
        balance = int(item["balance"]) / 1e18

        if item["status"] != "Open":
            # Other Statuses: "Waiting for commitment", "Closed", "Pending to close"
            # Ensures that nodes must have at least 1 open channel in to receive ct
            continue

        if source_peer_id not in peerid_address_aggbalance_links:
            peerid_address_aggbalance_links[source_peer_id] = {
                "source_node_address": source_address,
                "channels_balance": balance,
            }

        else:
            peerid_address_aggbalance_links[source_peer_id][
                "channels_balance"
            ] += balance

    return peerid_address_aggbalance_links

### Subgraph Data 

In [ ]:
subgraph_data = get_subgraph_data()
print(len(subgraph_data))

print_size = 10

for key, value in list(subgraph_data.items())[:print_size]:
    print(f"{key}: {value}")


### Topology data

In [ ]:
topology_data = get_unique_nodeAddress_peerId_aggbalance_links(api_host, api_key)

print(len(topology_data))

print_size = 5

for key, value in list(topology_data.items())[:print_size]:
    print(f"{key}: {value}")

In [ ]:
def merge_topology_subgraph(topology_dict: dict, subgraph_dict: dict):
    """
    Merge metrics and subgraph data with the unique peer IDs, addresses,
    balance links.
    :param: topology_dict: A dict mapping peer IDs to node addresses.
    :param: subgraph_dict: A dict containing subgraph data with safe address as the key.
    :returns: A dict with peer ID as the key and the merged information.
    """
    merged_result = {}

    # Merge based on peer ID with the channel topology as the baseline
    for peer_id, data in topology_dict.items():
        seen_in_subgraph = False

        source_node_address = data["source_node_address"]
        if source_node_address in subgraph_dict:
            subgraph_data = subgraph_dict[source_node_address]
            data["safe_address"] = subgraph_data["safe_address"]
            data["safe_balance"] = float(subgraph_data["wxHOPR_balance"])
            data["total_balance"] = data["channels_balance"] + data["safe_balance"]

            seen_in_subgraph = True
            # print(f"Source node address for {peer_id} found in subgraph")

        if seen_in_subgraph:
            merged_result[peer_id] = data

    return merged_result

In [ ]:
merged_data = merge_topology_subgraph(topology_data, subgraph_data)
print(len(merged_data))

print_size = 5

for key, value in list(merged_data.items())[:print_size]:
    print(f"{key}: {value}")

### Funds of Netwatchers 

In [ ]:
# Netwatcher 1
node_1_funds = merged_data["12D3KooWH9rfYNKMkNncYJxS7BH41ThPZUYe3FNkbfmJAa4n5r3x"]['total_balance']
print(node_1_funds)

In [ ]:
# Netwatcher 2
node_2_funds = merged_data["12D3KooWGyY39vD8J2VGEDjTCD3eEyvV4YrnKM9NCQa6SYJKczrR"]['total_balance']
print(node_2_funds)

In [ ]:
# Netwatcher 3
node_3_funds = merged_data["12D3KooWL16nW1Z2dLvyZWzr9ZZwoLTeuSfaKSeX2BjucHwSoEwJ"]['total_balance']
print(node_3_funds)

In [ ]:
# Netwatcher 4
node_4_funds = merged_data["12D3KooWNYi2kG5cdeEUBvjemZRUkPVmAeXsSGVrX9QHnEiMfh8w"]['total_balance']
print(node_4_funds)

### Relayed Token Calculation 

In [ ]:
ct_funding_received = 200000 # Funding transactions that the ct-app receives (should be automated by getting that information directly from the chain)
funds_in_ct_app = node_1_funds + node_2_funds + node_3_funds + node_4_funds
distributed_rewards = ct_funding_received - funds_in_ct_app
print(distributed_rewards)

In [ ]:
airdropped = 5*25000 # Get this number from Andrius
total_rewards = airdropped + distributed_rewards
print(total_rewards)